In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import iteround

In [2]:
G = nx.DiGraph()
G.add_edges_from([(0,1), (1,0), (1,2), (2,1)])
N = len(G.nodes)
E = len(G.edges)
c = np.array([1]*E)
p = np.array([100]*N)
vehicles = np.array([100, 50, 50])
a = np.array([0.3, 0.6, 0.1])

In [3]:

edge2index, index = {}, 0
for edge in G.edges():
    edge2index[edge] = index
    index += 1

# outbound edges
out_edges, in_edges = {}, {}
for n in range(N):
    edges = G.out_edges(n)
    out_edges[n] = [edge2index[e] for e in edges]
    edges = G.in_edges(n)
    in_edges[n] = [edge2index[e] for e in edges]

print(out_edges)
print(in_edges)

{0: [0], 1: [1, 2], 2: [3]}
{0: [1], 1: [0, 3], 2: [2]}


In [4]:
target = a*sum(vehicles)
target = iteround.saferound(target,0)
target = [int(t) for t in target]


In [5]:
print(N, E)
print(vehicles)
print(target)

3 4
[100  50  50]
[60, 120, 20]


In [6]:
# instantiate A, B and b_orig matrix

A = np.zeros((4*N, E))
B = np.zeros((4*N, N))
b_orig = np.zeros((4*N, 1))

row = 0
for n in range(N):
    for e in out_edges[n]:
        A[row, e] = 1
    b_orig[row,0] = vehicles[n]
    row += 1

for n in range(N):
    for e in in_edges[n]:
        A[row, e] = 1
        A[row+1, e] = -1
        A[row+2, e] = -1
    for e in out_edges[n]:
        A[row, e] = -1
        A[row+1, e] = 1
        A[row+2, e] = 1
    B[row, n] = -1
    B[row+1, n] = -1
    b_orig[row, 0] = -vehicles[n]+target[n]
    b_orig[row+1, 0] = vehicles[n]-target[n]
    b_orig[row+2, 0] = vehicles[n]
    row += 3

In [7]:
"""
Primal problem (forward run given the action a_i)
"""

m = gp.Model("forward-problem")

x = m.addVars(E, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
z = m.addVars(N, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="z")

# Set objective: maximize x
obj = 0
for e in range(E):
    obj += c[e] * x[e]
for n in range(N):
    obj += z[n] * p[n]
    
m.setObjective(obj, GRB.MINIMIZE)


for i in range(4*N):
    
    term1 = sum(A[i, j] * x[j] for j in range(E))
    term2 = sum(B[i, j] * z[j] for j in range(N))
    m.addConstr(term1 + term2 <= b_orig[i,0])

m.write('forward.lp')
m.optimize()
forward_objective = m.objVal
m.printAttr('x')
decision = m.getAttr('x')
print(decision)


# print dual variables

# Get the dual variables
dual_values = m.getAttr('Pi', m.getConstrs())
# Print the dual values

l_optimal = []
for constr, dual_value in zip(m.getConstrs(), dual_values):
#     print(f"Dual value for constraint {constr.ConstrName}: {dual_value}")
    l_optimal.append(dual_value)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-02
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 12 rows, 7 columns and 34 nonzeros
Model fingerprint: 0x28a936e1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 7 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.750000e+01   0.000000e+00      0s
       2    7.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.000000000e+01

    Variable            x 
-------------------------
        x[0]           40 
        x[3]           30 
[40.0, 0.0, 0.0, 30.0, 0.0, 

In [8]:
# Optimal flows
x_star = np.zeros(E)
for i in range(E):
    x_star[i] = x[i].X
total_vehicles = sum(vehicles)

print('optimal x: ', x_star)
print('b_orig: ',b_orig)
print('duals lambda : ',l_optimal)

optimal x:  [40.  0.  0. 30.]
b_orig:  [[100.]
 [ 50.]
 [ 50.]
 [-40.]
 [ 40.]
 [100.]
 [ 70.]
 [-70.]
 [ 50.]
 [-30.]
 [ 30.]
 [ 50.]]
duals lambda :  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]


In [9]:
# solve the dual problem!

m = gp.Model("dual-problem")

l = m.addVars(4*N, lb=-1*GRB.INFINITY, ub=0.0, vtype=GRB.CONTINUOUS, name="lambda")

obj = 0
for i in range(4*N):
    obj += b_orig[i,0] * l[i]

m.setObjective(obj, GRB.MAXIMIZE)

row, column = A.shape

for e in range(E):
    lhs = 0
    for r in range(row):
        lhs += A[r,e] * l[r]
    m.addConstr(lhs <= c[e])
    
for n in range(N):
    lhs = 0
    for r in range(row):
        lhs += B[r,n] * l[r]
    m.addConstr(lhs <= p[n])    

m.optimize()
m.printAttr('x')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 7 rows, 12 columns and 34 nonzeros
Model fingerprint: 0x43f19f4d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 34 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4000000e+32   7.000000e+30   1.400000e+02      0s
       5    7.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.000000000e+01

    Variable            x 
-------------------------
   lambda[3]         -0.5 
   lambda[4]        -99.5 
   lambda[7]         -100 
   lambda[9]         -0.5 
  lambda[10]        -99.5 


In [10]:
# Using an example solution
# Use it as an input for inverse optimization

m = gp.Model("inverse-problem")

eps = m.addVar(lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='eps')
z = m.addVars(N, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="z")
b = m.addVars(4*N, lb=-1*GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")
l = m.addVars(4*N, lb=-1*GRB.INFINITY, ub=0.0, vtype=GRB.CONTINUOUS, name="lambda")

 
regularizer = 0

# duality gap
m.addConstr(sum(c[i]*x_star[i] for i in range(E)) + sum(p[i] * z[i] for i in range(N)) - eps == sum(b[i]*l[i] for i in range(4*N)))

# Ax < b part
# vehicle outflow constraint
row, column = A.shape
for r in range(row):
    term1 = sum(A[r, j] * x_star[j] for j in range(E))
    term2 = sum(B[r, j] * z[j] for j in range(N))
    m.addConstr(term1 + term2 <= b[r])
    
# constraints on b
for n in range(N):
    m.addConstr(b[n] == vehicles[n])

index = 0
r = N
while r < 4*N:
    m.addConstr(b[r] ==  -1* b[r+1])
    m.addConstr(b[r+2] == vehicles[index])
    regularizer += np.random.rand()*1e-5*b[r]
    index += 1
    r += 3

for e in range(E):
    lhs = 0
    for r in range(row):
        lhs += A[r,e] * l[r]
    m.addConstr(lhs <= c[e])
    
for n in range(N):
    lhs = 0
    for r in range(row):
        lhs += B[r,n] * l[r]
    m.addConstr(lhs <= p[n])
    
m.setObjective(eps, GRB.MINIMIZE)
# m.setObjective(eps+regularizer, GRB.MINIMIZE)


m.Params.Presolve = 0
m.Params.Numericfocus = 3
m.Params.Method = 0
m.Params.BarHomogeneous = 1
m.Params.NonConvex = 2
# m.Params.DualReductions = 0
# m.reset()
# m.Params.TimeLimit = 5

m.optimize()
# m.computeIIS()
# m.write("iis.ilp")



# m.printAttr('x')
# decision = m.getAttr('x')
# print(decision)
m.write('inverse-withA.lp')

# for v in m.getVars():
#     print(f"{v.VarName} = {v.X}")
    
# extracting the solution

estimate_a = np.zeros(N)
for i in range(N):
    estimate_a[i] = b[N+3*i].X + vehicles[i]
    
estimate_a = estimate_a / sum(estimate_a)
print("Estimate for a : ", estimate_a)
print("True a: ", a)
# b_extracted = [b[i].X for i in range(4*N)]
# z_extracted = [z[i].X for i in range(N)]

Set parameter Presolve to value 0
Set parameter NumericFocus to value 3
Set parameter Method to value 0
Set parameter BarHomogeneous to value 1
Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 28 rows, 28 columns and 64 nonzeros
Model fingerprint: 0x0cc2c62a
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
  QRHS range       [7e+01, 7e+01]

Continuous model is non-convex -- solving as a MIP

Variable types: 35 continuous, 6 integer (0 binary)

Root relaxation: objective 0.000000e+00, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

In [11]:
# """
# Validating the inverse solution
# """

# m = gp.Model("forward-problem")

# x = m.addVars(E, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
# # z = m.addVars(N, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="z")

# # Set objective: maximize x
# obj = 0
# for e in range(E):
#     obj += c[e] * x[e]
# for n in range(N):
#     obj += z_extracted[n] * p[n]
    
# m.setObjective(obj, GRB.MINIMIZE)


# for i in range(4*N):
    
#     term1 = sum(A[i, j] * x[j] for j in range(E))
#     term2 = sum(B[i, j] * z_extracted[j] for j in range(N))
#     m.addConstr(term1 + term2 <= b_extracted[i])

    
# m.optimize()
# m.printAttr('x')
# decision = m.getAttr('x')
# print(decision)


In [12]:
# # checking for feasibility

# # Using an example solution
# # Use it as an input for inverse optimization

# m = gp.Model("inverse-problem")

# eps = m.addVar(lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='eps')
# z = m.addVars(N, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="z")
# b = m.addVars(4*N, vtype=GRB.CONTINUOUS, name="b")
# l = m.addVars(4*N, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="lambda")
              
              
# m.setObjective(eps, GRB.MINIMIZE)

# # duality gap
# m.addConstr(sum(c[i]*x_star[i] for i in range(E)) + sum(p[i] * z[i] for i in range(N)) - eps == sum(b[i]*l[i] for i in range(4*N)))

# # Ax < b part
# # vehicle outflow constraint
# row, column = A.shape
# for r in range(row):
#     term1 = sum(A[r, j] * x_star[j] for j in range(E))
#     term2 = sum(B[r, j] * z[j] for j in range(N))
#     m.addConstr(term1 + term2 <= b[r])
    
# # constraints on b
# # for n in range(N):
# #     m.addConstr(b[n] == vehicles[n])

# # index = 0
# # while row < 4*N:
# #     m.addConstr(b[row] ==  -1* b[row+1])
# #     m.addConstr(b[row+2] == vehicles[index])
# #     index += 1
# #     row += 3

# for e in range(E):
#     lhs = 0
#     for r in range(row):
#         lhs += A[r,e] * l[r]
#     m.addConstr(lhs <= c[e])
    
# for n in range(N):
#     lhs = 0
#     for r in range(row):
#         lhs += B[r,n] * l[r]
#     m.addConstr(lhs <= p[n])

# m.Params.NonConvex = 2
# m.optimize()
# m.printAttr('x')
# decision = m.getAttr('x')
# print(decision)
# m.write('inverse-withA.lp')

# for v in m.getVars():
#     print(f"{v.VarName} = {v.X}")
    
# # extracting the solution

# estimate_a = np.zeros(N)
# for i in range(N):
#     estimate_a[i] = b[N+3*i].X + vehicles[i]
    
# estimate_a = estimate_a / sum(estimate_a)
# print("Estimate for a : ", estimate_a)
# b_extracted = [b[i].X for i in range(4*N)]
# z_extracted = [z[i].X for i in range(N)]